# Build LDA model with sklearn

In [1]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

In [2]:
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

In [3]:
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# open title data
data_path = "/mount/studenten/knowledge-discovery/workspaces-no-backup/lemonwater/data/"
df_title = pd.read_json(data_path + 'covid19.0320.title.json')
df_title

,title,idx
0,backcalculating the incidence of infection wit...,0
1,preliminary identification of potential vaccin...,1
2,clinical medicine real-time estimation of the ...,2
3,a pneumonia outbreak associated with a new cor...,3
4,clinical medicine intense surveillance is vita...,4
...,...,...
29310,title systematic review and meta-analysis of p...,29310
29311,demographic science aids in understanding the ...,29311
29312,understanding epidemic data and statistics: a ...,29312
29313,effects of temperature variation and humidity ...,29313


In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/users0/changwn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# Write a function to perform lemmatize and stem preprocessing steps on the data set.
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) # lemmatize, pos
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result
lemmatize_stemming("rain")

'rain'

In [7]:
doc_sample = df_title[df_title['idx'] == 4310].values[0][0] # dict, value, text

# original document:
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)

# tokenized and lemmatized document
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))


original document: 
['malaria', 'from', 'hyperendemicity', 'to', 'elimination', 'in', 'hekou', 'county', 'on', 'china-', 'vietnam', 'border:', 'an', 'ecological', 'study']


 tokenized and lemmatized document: 
['malaria', 'hyperendem', 'elimin', 'hekou', 'counti', 'china', 'vietnam', 'border', 'ecolog', 'studi']


In [8]:
# pandas.Series.map() --> pandas.core.series.Series
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html
processed_docs = df_title['title'].map(preprocess)

In [9]:
print("Type of processed_docs: ", type(processed_docs))
processed_docs[:10]

Type of processed_docs:  <class 'pandas.core.series.Series'>


0    [backcalcul, incid, infect, covid, diamond, pr...
1    [preliminari, identif, potenti, vaccin, target...
2    [clinic, medicin, real, time, estim, risk, dea...
3    [pneumonia, outbreak, associ, coronavirus, pro...
4    [clinic, medicin, intens, surveil, vital, prev...
5    [clinic, medicin, optim, method, forecast, con...
6    [subunit, vaccin, emerg, pathogen, human, coro...
7    [clinic, medicin, incub, period, epidemiolog, ...
8                   [virus, return, coronavirus, ncov]
9    [clinic, medicin, rate, novel, coronavirus, nc...
Name: title, dtype: object

In [10]:
pd_list = []

for sent in processed_docs:
    #print(' '.join(sent))
    pd_list.append(' '.join(sent))

pd_list

['backcalcul incid infect covid diamond princess',
 'preliminari identif potenti vaccin target covid coronavirus sar base sar immunolog studi',
 'clinic medicin real time estim risk death novel coronavirus covid infect infer export case',
 'pneumonia outbreak associ coronavirus probabl origin',
 'clinic medicin intens surveil vital prevent sustain transmiss locat',
 'clinic medicin optim method forecast confirm case covid china',
 'subunit vaccin emerg pathogen human coronavirus',
 'clinic medicin incub period epidemiolog characterist novel coronavirus infect right truncat statist analysi public avail case data',
 'virus return coronavirus ncov',
 'clinic medicin rate novel coronavirus ncov infect estim japanes passeng data evacu flight',
 'polit diseas epidem compar analysi sar zika ebola outbreak',
 'statist base predict coronavirus epidem spread mainland china',
 'consensus statement speci sever acut respiratori syndrom relat coronavirus classifi ncov name sar coronavirida studi gro

<h2>Create the Document-Word matrix</h2>

In [11]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(pd_list)

<h2>Check the Sparsicity</h2>

In [12]:
data_dense = data_vectorized.todense()
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.2767548112790298 %


In [13]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=8,            # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


<h2>Diagnose model performance with perplexity and log-likelihood</h2>

In [14]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -1525209.897779037
Perplexity:  1094.4209825485832
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 8,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


<h2>GridSearch the best LDA model</h2>

In [16]:
# Define Search Param
search_params = {'n_components': [6, 8, 10, 12], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

/usr/lib64/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 

<h2>See the best topic model and its parameters</h2>

In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

<h2>Compare LDA Model Performance Scores</h2>

In [ ]:
# Get Log Likelyhoods from Grid Search Output
n_components = [6, 8, 10, 12]
log_likelyhoods_5 = [round(gscore.mean_validation_score) for gscore in model.cv_results_ 
                     if gscore.parameters['learning_decay']==0.5]
log_likelyhoods_7 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ 
                     if gscore.parameters['learning_decay']==0.7]
log_likelyhoods_9 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ 
                     if gscore.parameters['learning_decay']==0.9]

In [ ]:
# Show graph
plt.figure(figsize=(12, 8))
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.plot(n_topics, log_likelyhoods_9, label='0.9')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

<h2>See the dominant topic in each document</h2>

In [45]:
# Document - Topics distribution 
data = list(df_title["title"])
best_lda_model = lda_model

# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)


# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

df_document_topic_title = df_document_topic
df_document_topic_title['title'] = data ##
df_document_topic_title = df_document_topic.head(15)
df_document_topic_title



,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,dominant_topic,title
Doc0,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.85,7,backcalculating the incidence of infection wit...
Doc1,0.10,0.01,0.76,0.01,0.01,0.01,0.01,0.09,2,preliminary identification of potential vaccin...
Doc2,0.01,0.01,0.26,0.23,0.01,0.01,0.01,0.47,7,clinical medicine real-time estimation of the ...
Doc3,0.02,0.02,0.02,0.02,0.19,0.02,0.02,0.70,7,a pneumonia outbreak associated with a new cor...
Doc4,0.01,0.76,0.01,0.01,0.01,0.01,0.01,0.15,1,clinical medicine intense surveillance is vita...
Doc5,0.01,0.01,0.01,0.62,0.01,0.01,0.01,0.30,3,clinical medicine optimization method for fore...
Doc6,0.02,0.32,0.02,0.02,0.57,0.02,0.02,0.02,4,subunit vaccines against emerging pathogenic h...
Doc7,0.01,0.13,0.01,0.01,0.01,0.01,0.01,0.83,7,clinical medicine incubation period and other ...
Doc8,0.03,0.03,0.03,0.03,0.58,0.03,0.03,0.27,4,viruses return of the coronavirus: 2019-ncov
Doc9,0.01,0.10,0.01,0.01,0.01,0.12,0.01,0.74,7,clinical medicine the rate of underascertainme...


In [60]:
pd.set_option('display.max_colwidth', -1)
print(df_document_topic_title["title"][:15])

Doc0     backcalculating the incidence of infection with covid-19 on the diamond princess                                                                                                                                       
Doc1     preliminary identification of potential vaccine targets for the covid-19 coronavirus (sars-cov-2) based on sars-cov immunological studies                                                                              
Doc2     clinical medicine real-time estimation of the risk of death from novel coronavirus (covid-19) infection: inference using exported cases                                                                                
Doc3     a pneumonia outbreak associated with a new coronavirus of probable bat origin                                                                                                                                          
Doc4     clinical medicine intense surveillance is vital for preventing sustained transmission in ne

In [55]:
count = 0
for topic in df_document_topic["dominant_topic"]:
    count += 1
    if topic == 1:
        print(df_document_topic["title"][count])
    if count == 100:
        break

clinical medicine optimization method for forecasting confirmed cases of covid-19 in china
the novel coronavirus outbreak in wuhan, china
quantifying the association between domestic travel and the exportation of novel coronavirus (2019-ncov) cases from wuhan, china in 2020: a correlational analysis
clinical medicine estimation of the transmission risk of the 2019-ncov and its implication for public health interventions
subversion of cellular autophagosomal machinery by rna viruses
individual sequences in large sets of gene sequences may be distinguished efficiently by combinations of shared sub-sequences
journal of inflammation cxcr2 is critical for dsrna-induced lung injury: relevance to viral lung infection
respiratory research persistence of lung inflammation and lung cytokines with high-resolution ct abnormalities during recovery from sars
phosphatidylserine treatment relieves the block to retrovirus infection of cells expressing glycosylated virus receptors david a coil 1,2 and a

In [38]:
# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

TypeError: ("'>' not supported between instances of 'str' and 'float'", 'occurred at index title')

<h2>Review topics distribution across documents</h2>

In [19]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,0,6308
1,4,4164
2,1,3878
3,6,3733
4,5,3314
5,3,3288
6,2,2735
7,7,1895


<h2>Visualize the LDA model with pyLDAvis</h2>

In [20]:
pyLDAvis.enable_notebook()
# This is apply on sklearn best lda model (select from gesim result), topic = 8
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/home/users0/changwn/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4     -156.050995  253.896942       1        1  15.382437
6      105.162537  -46.559280       2        1  13.838564
3      173.813232 -261.729279       3        1  13.294755
1      312.827026   42.245800       4        1  12.876695
5      -92.090408 -223.994080       5        1  12.780279
2      -83.250290   39.574284       6        1  11.652699
0      110.335243  218.685135       7        1  10.468800
7     -292.986511  -45.538967       8        1   9.705772, topic_info=     Category         Freq         Term        Total  loglift  logprob
2006  Default  2860.000000  respiratori  2860.000000  30.0000  30.0000
2501  Default  5471.000000        virus  5471.000000  29.0000  29.0000
354   Default  2071.000000         cell  2071.000000  28.0000  28.0000
1153  Default  3841.000000       infect  3841.000000  27.0000  27.0000
1163  Default  1458.000000    influenza  1458.000000  26.0000  26.0000
...       ...          ...          ...          ...      ...      ...
792    Topic8   359.124806       epidem   814.907647   1.5130  -4.0759
1571   Topic8   305.904114        novel   964.223772   1.1844  -4.2363
415    Topic8   211.187169       clinic   948.549413   0.8303  -4.6068
518    Topic8   237.956450  coronavirus  2636.187851  -0.0726  -4.4875
108    Topic8   173.604826         anim   351.439774   1.6272  -4.8028

[395 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
20        6  0.997691    acid
21        8  0.993011  acquir
25        2  0.765026   activ
25        5  0.234460   activ
26        1  0.996442    acut
...     ...       ...     ...
2537      4  0.990496  worker
2539      7  0.997573   world
2541      8  0.995115   wuhan
2550      6  0.994007    zika
2554      7  0.991415  zoonot

[445 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 7, 4, 2, 6, 3, 1, 8])

In [ ]:
pyLDAvis.enable_notebook()
# This is apply on sklearn best lda model, topic = 10
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

<h2>See the Topic’s keywords</h2>

In [21]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,abbrevi,abdulaziz,abil,abnorm,abort,absenc,absorpt,abstract,abund,academ,...,yield,yolk,young,zealand,zika,zinc,zoonos,zoonosi,zoonot,zoster
Topic0,0.125227,0.125006,0.125154,0.125491,0.125211,0.125104,9.452364,0.125160,0.125083,0.125056,...,0.125046,0.125246,0.125163,16.260408,0.125131,26.979663,0.125071,0.125081,105.640282,0.125094
Topic1,0.125182,12.572965,0.125042,0.125108,0.125202,0.125077,0.125008,0.125424,12.124956,15.470725,...,0.125048,0.125030,0.125073,0.125246,0.125180,0.125038,0.125366,0.125392,0.125296,0.125011
Topic2,0.125235,0.125047,19.412583,0.125086,0.125210,0.125097,0.125065,30.125458,0.125195,0.125032,...,0.125162,0.125071,0.125069,0.125053,151.808601,0.125118,0.125093,0.125102,0.125097,0.125043
Topic3,0.125052,0.125055,0.125103,16.939372,0.125125,39.421172,0.125005,0.125132,0.125151,0.125089,...,6.108408,18.224764,0.125107,0.125161,0.125151,0.125097,58.451355,0.125075,0.125189,0.125068
Topic4,0.125225,0.125000,0.125091,0.125107,0.125196,0.125192,0.125042,0.125141,0.125101,0.125011,...,10.221394,0.125057,71.255817,0.125068,0.125119,0.125032,0.125114,0.125133,0.125111,0.125030


<h2>Get the top 15 keywords each topic</h2>

In [22]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,immun,vaccin,effect,respons,diseas,felin,manag,model,protect,mice,anti,enhanc,mechan,therapi,earli
Topic 1,diseas,health,emerg,infecti,control,care,public,studi,prevent,global,review,surveil,medicin,outbreak,hospit
Topic 2,sar,detect,apo,coronavirus,molecular,base,function,target,receptor,assay,develop,time,bind,analysi,acid
Topic 3,porcin,syndrom,respiratori,antibodi,coronavirus,patient,virus,middl,east,risk,review,infect,diarrhea,epidem,product
Topic 4,infect,respiratori,virus,influenza,viral,acut,human,sever,host,children,associ,coronavirus,mer,syndrom,diseas
Topic 5,virus,express,gene,genom,infect,sequenc,viral,inhibit,infecti,strain,coronavirus,protein,antivir,bronchiti,recombin
Topic 6,cell,protein,virus,activ,structur,human,induc,like,role,interact,coronavirus,type,interferon,regul,hepat
Topic 7,covid,transmiss,analysi,china,pneumonia,epidemiolog,case,data,outbreak,epidem,dynam,licens,novel,network,calv


<h2>Predict the topics for a new piece of text</h2>

In [ ]:
# Define function to predict topic for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))

    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)

    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["Some text about christianity and bible"]
topic, prob_scores = predict_topic(text = mytext)
print(topic)

<h2>Cluster documents that share similar topics and plot</h2>

In [ ]:
# Construct the k-means clusters
from sklearn.cluster import KMeans
clusters = KMeans(n_clusters=15, random_state=100).fit_predict(lda_output)

# Build the Singular Value Decomposition(SVD) model
svd_model = TruncatedSVD(n_components=2)  # 2 components
lda_output_svd = svd_model.fit_transform(lda_output)

# X and Y axes of the plot using SVD decomposition
x = lda_output_svd[:, 0]
y = lda_output_svd[:, 1]

# Weights for the 15 columns of lda_output, for each component
print("Component's weights: \n", np.round(svd_model.components_, 2))

# Percentage of total information in 'lda_output' explained by the two components
print("Perc of Variance Explained: \n", np.round(svd_model.explained_variance_ratio_, 2))

In [ ]:
# Plot
plt.figure(figsize=(12, 12))
plt.scatter(x, y, c=clusters)
plt.xlabel('Component 2')
plt.xlabel('Component 1')
plt.title("Segregation of Topic Clusters", )

<h2>Get similar documents for any given piece of text</h2>

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

nlp = spacy.load('en', disable=['parser', 'ner'])

def similar_documents(text, doc_topic_probs, documents = data, nlp=nlp, top_n=5, verbose=False):
    topic, x  = predict_topic(text)
    dists = euclidean_distances(x.reshape(1, -1), doc_topic_probs)[0]
    doc_ids = np.argsort(dists)[:top_n]
    if verbose:        
        print("Topic KeyWords: ", topic)
        print("Topic Prob Scores of text: ", np.round(x, 1))
        print("Most Similar Doc's Probs:  ", np.round(doc_topic_probs[doc_ids], 1))
    return doc_ids, np.take(documents, doc_ids)

In [ ]:
# Get similar documents
mytext = ["Some text about christianity and bible"]
doc_ids, docs = similar_documents(text=mytext, doc_topic_probs=lda_output, documents = data, top_n=1, verbose=True)
print('\n', docs[0][:500])